### **Import Dependencies**

In [ ]:
import json
from transformers import GPT2Tokenizer, GPT2LMHeadModel, AdamW, get_linear_schedule_with_warmup
from torch.utils.data import Dataset, DataLoader
import torch


### **Load Model**

In [ ]:
import json

dataset_path = "/content/drive/MyDrive/chatbot_models/ankita/intent.json"
with open(dataset_path, "r") as file:
    data = json.load(file)

intents = data["intents"]
questions = []
responses = []
for intent in intents:
    for pattern in intent["patterns"]:
        questions.append(pattern)
        # Append each response individually
        for response in intent["responses"]:
            responses.append(response)


In [ ]:
import json
from transformers import GPT2Tokenizer, GPT2LMHeadModel, AdamW, get_linear_schedule_with_warmup
from torch.utils.data import Dataset, DataLoader
import torch

# Load dataset from JSON file
dataset_path = "/content/drive/MyDrive/chatbot_models/ankita/intent.json"
with open(dataset_path, "r") as file:
    data = json.load(file)

intents = data["intents"]
questions = []
responses = []
for intent in intents:
    for pattern in intent["patterns"]:
        questions.append(pattern)
        for response in intent["responses"]:
            responses.append(response)

# Tokenization
tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
tokenizer.pad_token = tokenizer.eos_token
inputs_questions = tokenizer(questions, padding=True, truncation=True, return_tensors="pt", max_length=128)
inputs_responses = tokenizer(responses, padding=True, truncation=True, return_tensors="pt", max_length=128)



/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

### **Model Training**

In [ ]:
import torch
from torch.utils.data import Dataset, DataLoader
from transformers import GPT2Tokenizer, GPT2LMHeadModel, AdamW, get_linear_schedule_with_warmup
class CustomDataset(Dataset):
    def __init__(self, questions, responses, tokenizer, max_length):
        self.questions = questions
        self.responses = responses
        self.tokenizer = tokenizer
        self.max_length = max_length

    def __len__(self):
        return len(self.questions)

    def __getitem__(self, idx):
        question = self.questions[idx]
        response = self.responses[idx]

        encoding = self.tokenizer(question, response,
                                  padding='max_length',
                                  truncation=True,
                                  max_length=self.max_length,
                                  return_tensors='pt')

        input_ids = encoding['input_ids'].flatten()
        attention_mask = encoding['attention_mask'].flatten()
        labels = encoding['input_ids'].flatten()  # Use input_ids as labels for language modeling task

        return {
            "input_ids": input_ids,
            "attention_mask": attention_mask,
            "labels": labels
        }



# Load dataset and initialize tokenizer
dataset_path = "//content/drive/MyDrive/chatbot_models/ankita/intent.json"
with open(dataset_path, "r") as file:
    data = json.load(file)

intents = data["intents"]
questions = []
responses = []
for intent in intents:
    for pattern in intent["patterns"]:
        questions.append(pattern)
        for response in intent["responses"]:
            responses.append(response)

tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
tokenizer.pad_token = tokenizer.eos_token

# Create dataset
max_length = 128
dataset = CustomDataset(questions, responses, tokenizer, max_length)

# Create data loader
batch_size = 8
data_loader = DataLoader(dataset, batch_size=batch_size, shuffle=True)

# Define GPT-2 model and optimizer
model = GPT2LMHeadModel.from_pretrained("gpt2")
optimizer = AdamW(model.parameters(), lr=5e-5)
scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=0, num_training_steps=len(data_loader))

# Define training loop
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

num_epochs = 3
for epoch in range(num_epochs):
    model.train()
    total_loss = 0
    for batch in data_loader:
        input_ids = batch["input_ids"].to(device)
        attention_mask = batch["attention_mask"].to(device)
        labels = batch["labels"].to(device)

        optimizer.zero_grad()
        outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs.loss
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
        optimizer.step()
        scheduler.step()

        total_loss += loss.item()

    avg_loss = total_loss / len(data_loader)
    print(f"Epoch {epoch+1}/{num_epochs}, Average Loss: {avg_loss:.4f}")


model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:521: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Epoch 1/3, Average Loss: 6.3213
Epoch 2/3, Average Loss: 3.7410
Epoch 3/3, Average Loss: 3.8612


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


### **Chatbot**

In [ ]:
# Function to chat with the bot
def chat_with_bot(model, tokenizer):
    model.eval()
    while True:
        user_input = input("You: ")
        if user_input.lower() == 'quit':
            print("Chatbot: Goodbye!")
            break
        # Tokenize user input
        user_input_ids = tokenizer.encode(user_input, return_tensors='pt')

        user_input_ids = user_input_ids.to(device)
        # Generate response
        bot_output = model.generate(user_input_ids, num_return_sequences=1, pad_token_id=tokenizer.eos_token_id)
        # Decode and print response
        bot_response = tokenizer.decode(bot_output[0], skip_special_tokens=True)
        print("Chatbot:", bot_response)

# Start chatting
chat_with_bot(model, tokenizer)

Let's chat! (type 'quit' to exit)
You: How can I contact customer support?
Chetana: Received a damaged item? Contact our support team for assistance with returns and replacements.
You: I need help with my account.
Chetana: To report a problem with your order, please contact our customer support team.
You: Who should I contact regarding a complaint?
Chetana: Interested in placing a pre-order? Contact our support team for assistance.
You: What's the best way to get help with a problem?
Chetana: Set preferences for order notifications by contacting our support team.
You: quit


### **Model Evaluation**

In [ ]:
# Define a function to evaluate the model
import nltk
from nltk.tokenize import word_tokenize
import json
import numpy as np
import torch

nltk.download('punkt')
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

def tokenize(sentence):
    return word_tokenize(sentence)

def bag_of_words(sentence, words):
    bag = np.zeros(len(words), dtype=np.float32)
    for word in sentence:
        if word in words:
            bag[words.index(word)] = 1
    return bag


def evaluate_model(model, test_data, intents, tokenizer, device):
    num_correct = 0
    total = 0

    for example in test_data:
        question = example['sentence']
        expected_tag = example['tag']
        # print(tags)
        # Concatenate the question with each class label
        inputs = [f"{question} {tag}" for tag in tags]

        # Tokenize the inputs
        encoding = tokenizer(inputs, padding=True, truncation=True, return_tensors="pt")
        input_ids = encoding['input_ids'].to(device)
        attention_mask = encoding['attention_mask'].to(device)

        # Forward pass
        with torch.no_grad():
            outputs = model(input_ids=input_ids, attention_mask=attention_mask)

        # Extract logits
        logits = outputs.logits

        # Calculate probabilities
        probabilities = torch.softmax(logits, dim=-1)

        # Choose the class with the highest probability
        predicted_indices = torch.argmax(probabilities, dim=-1)

        # Convert predicted_indices tensor to a list of lists
        predicted_indices_list = predicted_indices.tolist()


        # Get corresponding tags for each predicted index
        predicted_tags = []
        for index_list in predicted_indices_list:
            for index in index_list:
                if index >= 0 and index < len(tags):
                    predicted_tags.append(tags[index])
        # print(expected_tag, predicted_tags)
        # Check if any of the predicted tags match the expected tag
        if expected_tag in predicted_tags:

            num_correct += 1

        total += 1

    accuracy = num_correct / total
    print(f"Accuracy: {accuracy * 100:.2f}%")


# Load the test dataset
test_data = [
    {"sentence": "Hi there!", "tag": "greeting"},
    {"sentence": "Hello there!! How may I help you?", "tag": "greeting"},
    {"sentence": "How are you", "tag": "greeting"},
    {"sentence": "Hello there", "tag": "greeting"},
    {"sentence": "Hi there! How's everything been going with you lately?", "tag": "greeting"},
    {"sentence": "Hello there! Welcome back.", "tag": "greeting"},
    {"sentence": "Hello, What do you need today?", "tag": "greeting"},
    {"sentence": "Hello, Good morning?", "tag": "greeting"},
    {"sentence": "You're welcome! Happy to assist. Goodbye", "tag": "goodbye"},
    {"sentence": "Hello!! How is your day going?", "tag": "greeting"},
    {"sentence": "Bye. Have a good day", "tag":"goodbye"},
    {"sentence": "Hi, how is it going", "tag": "greeting"}

]

with open('/content/drive/MyDrive/chatbot_models/ankita/intent.json', 'r') as json_data:
    intents = json.load(json_data)

# Assuming your training data is stored in a variable called intents
all_words = []
for intent in intents['intents']:
    for pattern in intent['patterns']:
        words = tokenize(pattern)
        all_words.extend(words)

# Extracting unique tags from the intents data
tags = []
for intent in intents['intents']:
    tags.append(intent['tag'])


# Remove duplicates
all_words = list(set(all_words))


# Call the evaluate_model function
evaluate_model(model, test_data, intents, tokenizer, device)


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


Accuracy: 41.67%
